In [34]:
import numpy as np
import pandas as pd
import networkx as nx
from stellargraph import StellarGraph
from stellargraph.data import BiasedRandomWalk
from gensim.models import Word2Vec
import csv

In [4]:
#Get train.csv data
df = pd.read_fwf('train.csv', header=None)
df = df[0].str.split(',', expand=True) 

In [6]:
#Get ingredient id
df2 = pd.read_fwf('node_ingredient.csv', header=None)
idToNode = {}
for index , row in df2.iterrows():
    idToNode[index] = row[0]


In [7]:
len(idToNode)

6714

In [8]:
#Create dict of dict where the first key is ingredient id the value is another dict 
#with key are nodes he shares receipt with and value the number of receipt they have in common
ingredientAdjancy = {}
ingredientNbR = {}
for index , row in df.iterrows():
    for i in row.values:
        if i is not None and i.isdigit() and int(i) not in ingredientAdjancy.keys():
            ingredientAdjancy[int(i)] = {}
        for y in row.values:
            if i is not None and  y is not None and i.isdigit() and y.isdigit():
                if i != y:
                    if int(y) not in ingredientAdjancy[int(i)].keys():
                        ingredientAdjancy[int(i)][int(y)] = 1
                    else:
                        ingredientAdjancy[int(i)][int(y)] += 1
        if i is not None and i.isdigit():
            if int(i) not in ingredientNbR.keys():
                ingredientNbR[int(i)] = 1
            else:
                ingredientNbR[int(i)] += 1


In [13]:
len(ingredientAdjancy)

5860

In [22]:
with open('edgeListWeight.csv','w', newline='') as fh:
    writer = csv.writer(fh, delimiter=',')
    for i in ingredientAdjancy.keys():
        for y in ingredientAdjancy[i].keys():
            writer.writerow([i, y, ingredientAdjancy[i][y]])

In [23]:
with open('edgeList.csv','w', newline='') as fh:
    writer = csv.writer(fh, delimiter=',')
    for i in ingredientAdjancy.keys():
        for y in ingredientAdjancy[i].keys():
            writer.writerow([i, y])

In [27]:
Gw = nx.read_edgelist('edgeListWeight.csv', delimiter=',', nodetype=int, data=(('weight',float),))
G = nx.read_edgelist('edgeList.csv', delimiter=',', nodetype=int, data=(('weight',float),))

In [28]:
print(len(G.nodes()))
print(len((G.edges())))
nodes = list(G.nodes())


5860
353396


In [46]:
import os
os.getcwd()

'C:\\Users\\Felix\\Documents\\KAIST\\Courses\\Data Mining\\Project\\Code\\AI506-Project'

In [53]:
pd.read_fwf('Embedding/Embp1q2.csv', header=None, delimiter=" ")

,0
0,"2813,""[ 0.20561211 -0.23588583 0.28525352 0...."
1,-0.01796227 -0.11046874 -0.5681695 -0.0678984...
2,-0.12082586 -0.2529383 0.3063318 0.3963518...
3,0.04451386 0.19073167 0.32635474 0.44178614...
4,-0.20612109 0.2922662 -0.18655235 0.1226137...
...,...
66895,0.02855606 0.01496908 -0.00278561 0.00684646...
66896,-0.00035502 -0.02015509 0.04706188 -0.0076659...
66897,0.0042229 0.07924966 0.03077858 0.0288952 ...
66898,0.04514599 0.0133772 0.0227813 -0.05956929...


Some ingredient are not part of the graph (have no edge)

Use Node2vec -> first need to generate weighted random walk. Then use Word2Vec to generate the embedding.
Generate random walktake quite some time, I already generate 4 of them using different paramters that should adjust the balance between sutrucural and local importance. This might be improved by trying different set of hyperparameter. 

In [35]:
#Use StellarGraph library to generate the weighted random walk
GwS = StellarGraph.from_networkx(Gw)

In [36]:
rw = BiasedRandomWalk(GwS)

In [37]:
weighted_walks = rw.run(
        nodes=G.nodes(), 
        length=100,    # maximum length of a random walk
        n=10,          # number of random walks per root node
        p=1,         # Defines (unormalised) probability, 1/p, of returning to source node
        q=100,         # Defines (unormalised) probability, 1/q, for moving away from source node
        weighted=True, #for weighted random walks
        seed=42        # random seed fixed for reproducibility
    )

KeyboardInterrupt: 

In [ ]:
str(weighted_walks[1])

In [ ]:
#Save the random walk
with open('NodeRandomWalkp1q100.csv','w', newline='') as fh:
    writer = csv.writer(fh, delimiter=',')
    for i in range(len(G.nodes())):
        for y in range(10):
            writer.writerow([str(GwS.nodes()[i]), weighted_walks[i*10+y]])

In [28]:
#Import random walk
import ast
rp1q10 = pd.read_fwf('RW/WRandomWalkp1q2.csv', header=None, delimiter= " ")

rp1q10_list = []
for index, value in rp1q10.iterrows():
    t = False
    s = ""
    for i in value[0]:
        if i == "[":
            t = True
        if i == '"' and t:
            t = False
        if t:
            s+=i
    if s[len(s)-1] != "]":
        s+="]"
    rp1q10_list.append(ast.literal_eval(s)) 
  

In [29]:
from gensim.test.utils import common_texts
from gensim.models import Word2Vec

embNod2V = Word2Vec(sentences=rp1q10_list, vector_size=64, window=3, min_count=0, sg=1, workers=1, epochs=1)

In [30]:
len(embNod2V.wv)

5865

In [51]:
from numpy import dot
from numpy.linalg import norm
m=0
test = 6000
for i in Gw.nodes():
    if i != test:
        a = embNod2V.wv[test]
        b = embNod2V.wv[i]
        cos_sim = dot(a, b)/(norm(a)*norm(b))
        if cos_sim > m:
            vm = i
            m = cos_sim
print(m,vm)
print(idToNode[test]," is most similar to: ",idToNode[vm])

0.9917877 1529
min  is most similar to:  tamarind extract


In [52]:
fh = open('Embedding/Embp1q100.csv','w')
writer = csv.writer(fh)
for i in Gw.nodes():
    writer.writerow([i,embNod2V.wv[i]])
fh.close()

Second embedding method (not weighted)

In [70]:
from sklearn.decomposition import  TruncatedSVD

A = nx.adjacency_matrix(G) 

svd = TruncatedSVD(n_components=128)
X = svd.fit_transform(A);

Save different embedding size (You can find 32-64-128 in /embedding)

In [71]:
import csv

fh = open('Embedding/SVD128.csv','w')
writer = csv.writer(fh)

y = 0
for i in G.nodes():
    writer.writerow([i,X[y]]) 
    y += 1
    
fh.close()

In [74]:
test = 6
m = 0

for i in range(5860):
    if i != test:
        a = X[test]
        b = X[i]
        cos_sim = dot(a, b)/(norm(a)*norm(b))
        if cos_sim > m:
            vm = i
            m = cos_sim
print(m,vm)
print(idToNode[list(G.nodes())[test]]," is most similar to: ",idToNode[list(G.nodes())[vm]])

0.8458904945495505 264
black olives  is most similar to:  olives
